# **Data Acquisition**

- In this section, we utilize openFDA's Food API to explore adverse events related to FDA-regulated foods, dietary supplements, and cosmetics.
- Specifically, our focus is on the Soft Drink/Water industry.
- Due to API constraints, the maximum value for the limit parameter is currently set at 1000.

In [195]:
# import urllib library
from urllib.request import urlopen
import pandas as pd

# import json
import json
# store the URL in url as
# parameter for urlopen
url = "https://api.fda.gov/food/event.json?search=products.industry_name:%22Soft+Drink/Water%22&limit=1000" # Currently, the largest allowed value for the limit parameter is 1000.

# store the response of URL
response = urlopen(url)

# storing the JSON response
# from url in data
data_json = json.loads(response.read())

# print the json response
print(data_json)


{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service. Submission of an adverse event report does not constitute an admission that a product caused or contributed to an event. The information in these reports has not been scientifically or otherwise verified as to a cause and effect relationship and cannot be used to estimate incidence (occurrence rate) or to estimate risk.', 'terms': 'https://open.fda.gov/terms/', 'license': 'https://open.fda.gov/license/', 'last_updated': '2025-02-09', 'results': {'skip': 0, 'limit': 1000, 'total': 3476}}, 'results': [{'report_number': '2024-CFS-003668', 'outcomes': ['Other Outcome'], 'date_created': '20240410', 'reactions': ['Nausea', 'Rash', 'Throat irritation'], 'date_started': '20240404', 'consumer': {'age': '44', '

In [196]:
data_json.keys()
df = pd.DataFrame(data_json["results"])
df

,report_number,outcomes,date_created,reactions,date_started,consumer,products
0,2024-CFS-003668,[Other Outcome],20240410,"[Nausea, Rash, Throat irritation]",20240404,"{'age': '44', 'age_unit': 'year(s)', 'gender':...","[{'role': 'SUSPECT', 'name_brand': 'GATORADE F..."
1,183113,[Other Outcome],20150218,[MALAISE],20150110,{'gender': 'Male'},"[{'role': 'SUSPECT', 'name_brand': 'COCA-COLA ..."
2,2017-CFS-000711,[Other Outcome],20170907,"[Abdominal pain, Diarrhoea]",None,{},"[{'role': 'SUSPECT', 'name_brand': 'black cher..."
3,194654,[Other Outcome],20141009,"[ABNORMAL BEHAVIOUR, BURNING SENSATION, CARDIA...",20141007,{'gender': 'Female'},"[{'role': 'SUSPECT', 'name_brand': 'KIRKLAND S..."
4,69316,[Other Outcome],20040518,[NAUSEA],None,{'gender': 'Male'},"[{'role': 'SUSPECT', 'name_brand': 'PEPSI COLA..."
...,...,...,...,...,...,...,...
995,67354,[Other Outcome],20031209,[CHOKING],20031208,"{'age': '62', 'age_unit': 'year(s)', 'gender':...","[{'role': 'SUSPECT', 'name_brand': 'PEPSI COLA..."
996,66059,[Visited Emergency Room],20040123,"[TOOTH DISCOLOURATION, VOMITING]",None,{'gender': 'Female'},"[{'role': 'SUSPECT', 'name_brand': 'STEWARTS R..."
997,65420,"[Life Threatening, Hospitalization, Disability]",20040106,"[CHEST PAIN, DIZZINESS, EXTRASYSTOLES, HOSPITA...",20031101,"{'age': '33', 'age_unit': 'year(s)', 'gender':...","[{'role': 'SUSPECT', 'name_brand': 'COFFEE', '..."
998,95035,"[Hospitalization, Visited Emergency Room]",20070801,"[ATELECTASIS, CHEST PAIN, JOINT LOCK, JOINT ST...",20070731,"{'age': '23', 'age_unit': 'year(s)', 'gender':...","[{'role': 'SUSPECT', 'name_brand': 'ACE ENERGY..."


# **Data Cleaning and Handling Inconsistencies**

In this section, we will:
- Fix incorrect data formats (e.g., convert date from string to datetime).
- Correct the consumer and product columns, which contain data in nested dictionary format, by separating them into individual columns.
- Handle lists in the outcomes and reactions columns by creating dummy variables for each item in the list for further analysis.
- Remove missing values.
- Remove duplicate rows.
- Standardize all alphabets to lower case letters.

Lastly, we will show the summary of the unique values for each category.

In [197]:
# Ensure Proper Datetime Formatting
df['date_created'] = pd.to_datetime(df['date_created'], format='%Y%m%d', errors='coerce')
df['date_started'] = pd.to_datetime(df['date_started'], format='%Y%m%d', errors='coerce')

# Normalize the 'consumer' column
consumer_df = pd.json_normalize(df['consumer'])
df = pd.concat([df.drop(columns=['consumer']), consumer_df], axis=1)

# Separate the 'role' and 'name_brand' from the 'products' column
df['product_role'] = df['products'].apply(lambda x: x[0]['role'] if isinstance(x, list) and len(x) > 0 else None)
df['product_name_brand'] = df['products'].apply(lambda x: x[0]['name_brand'].lower() if isinstance(x, list) and len(x) > 0 and isinstance(x[0].get('name_brand'), str) else None)

df.drop(columns=['products'], inplace=True)

# Drop report_number column, product_role and age_unit column since they are not necessary for further analysis
df.drop(columns=['report_number'], inplace=True)
df.drop(columns=['age_unit'], inplace=True)
df.drop(columns=['product_role'], inplace=True)

# Since data are in list format, flatten the data
df['outcomes'] = df['outcomes'].apply(lambda x: x if isinstance(x, list) else [x]) \
                             .explode() \
                             .apply(lambda x: x.lower() if isinstance(x, str) else x) \
                             .reset_index(drop=True)

df['reactions'] = df['reactions'].apply(lambda x: x if isinstance(x, list) else [x]) \
                                 .explode() \
                                 .apply(lambda x: x.lower() if isinstance(x, str) else x) \
                                 .reset_index(drop=True)

# Create dummy variables for the 'outcomes' column
outcomes_dummies = pd.get_dummies(df['outcomes'], prefix='outcome')

# Create dummy variables for the 'reactions' column
reactions_dummies = pd.get_dummies(df['reactions'], prefix='reaction')

df = pd.concat([df, outcomes_dummies, reactions_dummies], axis=1)

# Remove missing values (only  288 items are left)
df = df.dropna()

# Reset the index for dataframe
df = df.reset_index(drop=True)

df

,outcomes,date_created,reactions,date_started,age,gender,product_name_brand,outcome_allergic reaction,outcome_death,outcome_disability,...,reaction_viral test positive,reaction_vision blurred,reaction_visual acuity reduced,reaction_visual disturbance,reaction_visual impairment,reaction_vomiting,reaction_vulvovaginal burning sensation,reaction_weight decreased,reaction_weight increased,reaction_wheezing
0,other outcome,2024-04-10,nausea,2024-04-04,44,Female,gatorade frost thirst quencher,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,other outcome,2014-10-02,abnormal behaviour,2014-09-29,56,Female,gatorade g2 orange drink,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,other serious or important medical event,2021-04-11,cardiac fibrillation,2021-04-11,45,Female,gatorade thirst quencher orange,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,required intervention,2016-07-19,hypoaesthesia,2016-07-16,33,Male,puraqua purified water,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,other serious or important medical event,2011-11-01,skin burning sensation,2011-03-14,70,Male,spring valley beta carotene 25000 iu,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,visited emergency room,2010-10-06,chest pain,2010-08-07,67,Male,canada dry tonic water,False,False,False,...,False,False,False,False,False,False,False,False,False,False
284,visited a health care provider,2003-12-09,palpitations,2003-12-08,62,Female,pepsi cola soft drink,False,False,False,...,False,False,False,False,False,False,False,False,False,False
285,other outcome,2004-01-06,hypoacusis,2003-11-01,33,Male,coffee,False,False,False,...,False,False,False,False,False,False,False,False,False,False
286,other serious or important medical event,2007-08-01,pyrexia,2007-07-31,23,Male,ace energy drink,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [198]:
initial_shape = df.shape
df = df.drop_duplicates()
print(f"Removed {initial_shape[0] - df.shape[0]} duplicate rows. New shape: {df.shape}")

print("Unique values after cleaning:")
print("Outcomes:", df['outcomes'].unique())
print("Reactions:", df['reactions'].unique())
print("Age:", df['age'].unique())
print("Gender:", df['gender'].unique())
print("Product Name Brand:", df['product_name_brand'].unique())

Removed 0 duplicate rows. New shape: (288, 265)
Unique values after cleaning:
Outcomes: ['other outcome' 'other serious or important medical event'
 'required intervention' 'visited a health care provider'
 'hospitalization' 'life threatening' 'visited emergency room'
 'other serious outcome' 'disability' 'death']
Reactions: ['nausea' 'abnormal behaviour' 'cardiac fibrillation' 'hypoaesthesia'
 'skin burning sensation' 'diarrhoea' 'glossodynia' 'abdominal pain'
 'hypotension' 'oropharyngeal swelling' 'eye pain' 'lymphadenopathy'
 'musculoskeletal pain' 'asthenia' 'convulsion' 'pain' 'panic attack'
 'vomiting' 'abdominal pain upper' 'oral discomfort' 'thermal burn'
 'burning sensation' 'jaundice' 'photophobia' 'gastritis' 'dizziness'
 'musculoskeletal disorder' 'abdominal discomfort' 'headache' 'fatigue'
 'blister' 'dry skin' 'rash papular' 'infection' 'grand mal convulsion'
 'gait disturbance' 'bronchitis' 'eye irritation' 'dyspnoea'
 'weight decreased' 'urticaria' 'throat irritation' 